In [66]:
%load_ext autoreload
%autoreload 2

from src.base_classes.omic_data_loader import OmicDataLoader
from src.baseline_models_benchmarks.data_manager import CatOmicDataManager

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [67]:
mrna_loader = OmicDataLoader(
    data_dir="mds_data/splits/mrna",
)
mirna_loader = OmicDataLoader(
    data_dir="mds_data/splits/mirna",
)

files
['train_0.csv', 'train_1.csv', 'train_2.csv', 'train_3.csv', 'train_4.csv']
['test_0.csv', 'test_1.csv', 'test_2.csv', 'test_3.csv', 'test_4.csv']
files
['train_0.csv', 'train_1.csv', 'train_2.csv', 'train_3.csv', 'train_4.csv']
['test_0.csv', 'test_1.csv', 'test_2.csv', 'test_3.csv', 'test_4.csv']


In [68]:
for fold_idx in range(5):

    train_df, test_df = mrna_loader.get_fold(fold_idx)

    print("fold: ", fold_idx)
    print(train_df["class"].value_counts(), test_df["class"].value_counts())

fold:  0
shape: (2, 2)
┌───────┬───────┐
│ class ┆ count │
│ ---   ┆ ---   │
│ i64   ┆ u32   │
╞═══════╪═══════╡
│ 0     ┆ 5     │
│ 1     ┆ 47    │
└───────┴───────┘ shape: (2, 2)
┌───────┬───────┐
│ class ┆ count │
│ ---   ┆ ---   │
│ i64   ┆ u32   │
╞═══════╪═══════╡
│ 0     ┆ 2     │
│ 1     ┆ 12    │
└───────┴───────┘
fold:  1
shape: (2, 2)
┌───────┬───────┐
│ class ┆ count │
│ ---   ┆ ---   │
│ i64   ┆ u32   │
╞═══════╪═══════╡
│ 0     ┆ 5     │
│ 1     ┆ 48    │
└───────┴───────┘ shape: (2, 2)
┌───────┬───────┐
│ class ┆ count │
│ ---   ┆ ---   │
│ i64   ┆ u32   │
╞═══════╪═══════╡
│ 0     ┆ 2     │
│ 1     ┆ 11    │
└───────┴───────┘
fold:  2
shape: (2, 2)
┌───────┬───────┐
│ class ┆ count │
│ ---   ┆ ---   │
│ i64   ┆ u32   │
╞═══════╪═══════╡
│ 0     ┆ 6     │
│ 1     ┆ 47    │
└───────┴───────┘ shape: (2, 2)
┌───────┬───────┐
│ class ┆ count │
│ ---   ┆ ---   │
│ i64   ┆ u32   │
╞═══════╪═══════╡
│ 0     ┆ 1     │
│ 1     ┆ 12    │
└───────┴───────┘
fold:  3
shape: (2, 2)
┌─

In [69]:
omic_data_loaders = {"mrna": mrna_loader, "mirna": mirna_loader}
odm = CatOmicDataManager(omic_data_loaders, n_splits=5)

odm.get_split(0)

(shape: (52, 400)
 ┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
 │ ENSG00000 ┆ ENSG00000 ┆ ENSG00000 ┆ ENSG00000 ┆ … ┆ hsa-miR-4 ┆ hsa-miR-5 ┆ hsa-miR-5 ┆ hsa-miR- │
 │ 088899    ┆ 181790    ┆ 138411    ┆ 138867    ┆   ┆ 54-3p     ┆ 48f-5p    ┆ 695       ┆ 29a-5p   │
 │ ---       ┆ ---       ┆ ---       ┆ ---       ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---      │
 │ f64       ┆ f64       ┆ f64       ┆ f64       ┆   ┆ f64       ┆ f64       ┆ f64       ┆ f64      │
 ╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
 │ 0.888473  ┆ 0.366753  ┆ 0.40207   ┆ 0.659703  ┆ … ┆ 0.844496  ┆ 0.445874  ┆ 0.338697  ┆ 0.081448 │
 │ 0.842566  ┆ 0.330205  ┆ 0.441948  ┆ 0.831243  ┆ … ┆ 0.814479  ┆ 0.0       ┆ 0.0       ┆ 0.230676 │
 │ 1.0       ┆ 0.039607  ┆ 0.097896  ┆ 0.852972  ┆ … ┆ 0.614929  ┆ 0.567646  ┆ 0.369975  ┆ 0.389797 │
 │ 0.893953  ┆ 0.280645  ┆ 0.265928  ┆ 0.837996  ┆ … ┆ 0.810841 

In [70]:
print(odm.train_y)
print(odm.test_y)

None
None


In [71]:
odm.get_split(1)

(shape: (53, 400)
 ┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
 │ ENSG00000 ┆ ENSG00000 ┆ ENSG00000 ┆ ENSG00000 ┆ … ┆ hsa-miR-2 ┆ hsa-miR-6 ┆ hsa-miR-6 ┆ hsa-miR- │
 │ 113552    ┆ 204252    ┆ 171067    ┆ 078804    ┆   ┆ 9b-3p     ┆ 808-3p    ┆ 60-3p     ┆ 337-3p   │
 │ ---       ┆ ---       ┆ ---       ┆ ---       ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---      │
 │ f64       ┆ f64       ┆ f64       ┆ f64       ┆   ┆ f64       ┆ f64       ┆ f64       ┆ f64      │
 ╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
 │ 0.101906  ┆ 0.67432   ┆ 0.553844  ┆ 0.427465  ┆ … ┆ 0.332964  ┆ 0.491693  ┆ 0.228302  ┆ 0.478208 │
 │ 0.211153  ┆ 0.79269   ┆ 0.443358  ┆ 0.538522  ┆ … ┆ 0.231004  ┆ 0.367655  ┆ 0.0       ┆ 0.463624 │
 │ 0.248448  ┆ 0.901163  ┆ 0.376432  ┆ 0.818353  ┆ … ┆ 0.567736  ┆ 0.200131  ┆ 0.0       ┆ 0.377795 │
 │ 0.0       ┆ 0.855781  ┆ 0.095358  ┆ 0.86714   ┆ … ┆ 0.390975 